#### Import Libraries

In [1]:
#*******________________________Code for Training DSD_Net______________________________*******

import torch
import torch.nn as nn
import os, glob
import random, csv
from torch.autograd import Variable

#import visdom
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm
import numpy as np
import pywt
import pywt.data

#### Custom Dataloader

In [2]:
import random
import numpy as np
import torch.utils.data as data
import utils.utils_image as util


class DatasetSR(data.Dataset):
   

    def __init__(self):
        super(DatasetSR, self).__init__()
        self.n_channels = 1
        self.sf = 1
        self.patch_size = 128
        self.L_size = self.patch_size // self.sf

        # ------------------------------------
        # get paths of L/H
        # ------------------------------------
        self.paths_H = util.get_image_paths('/notebooks/JPEG_HR_Gray/DIV2K_train_HR_Grayscale')    #The Path of Ground Truth Images
        self.paths_L = util.get_image_paths('/notebooks/IACNN_Detection_Files/Anchor/JPEG_10')     #The path of LR Images
        '''
        assert self.paths_H, 'Error: H path is empty.'
        if self.paths_L and self.paths_H:
            assert len(self.paths_L) == len(self.paths_H), 'L/H mismatch - {}, {}.'.format(len(self.paths_L), len(self.paths_H))
        '''
    def __getitem__(self, index):

        H_path = None
        # ------------------------------------
        # get H image
        # ------------------------------------
        L_path = self.paths_L[index]
        img_L = util.imread_uint(L_path, self.n_channels)
        img_L = util.uint2single(img_L)

        # ------------------------------------
        # modcrop
        # ------------------------------------
        

        # ------------------------------------
        # get L image
        # ------------------------------------
        if self.paths_H:
            # --------------------------------
            # directly load L image
            # --------------------------------
            H_path = self.paths_H[index % 800]
            img_H = util.imread_uint(H_path, self.n_channels)
            img_H = util.uint2single(img_H)
            img_H = util.modcrop(img_H, self.sf)
        else:
            # --------------------------------
            # sythesize L image via matlab's bicubic
            # --------------------------------
            H, W = img_H.shape[:2]
            img_L = util.imresize_np(img_H, 1 / self.sf, True)

        # ------------------------------------
        # if train, get L/H patch pair
        # ------------------------------------

        H, W, C = img_L.shape

        # --------------------------------
        # randomly crop the L patch
        # --------------------------------
        rnd_h = random.randint(0, max(0, H - self.L_size))
        rnd_w = random.randint(0, max(0, W - self.L_size))
        img_L = img_L[rnd_h:rnd_h + self.L_size, rnd_w:rnd_w + self.L_size]

        # --------------------------------
        # crop corresponding H patch
        # --------------------------------
        rnd_h_H, rnd_w_H = int(rnd_h * self.sf), int(rnd_w * self.sf)
        img_H = img_H[rnd_h_H:rnd_h_H + self.patch_size, rnd_w_H:rnd_w_H + self.patch_size]

        # --------------------------------
        # augmentation - flip and/or rotate
        # --------------------------------
        
        mode = random.randint(0, 7)
        img_L, img_H = util.augment_img(img_L, mode=mode), util.augment_img(img_H, mode=mode)
        
        # ------------------------------------
        # L/H pairs, HWC to CHW, numpy to tensor
        # ------------------------------------
        img_H, img_L = util.single2tensor3(img_H), util.single2tensor3(img_L)

        if L_path is None:
            L_path = H_path

        return {'L': img_L, 'H': img_H, 'L_path': L_path, 'H_path': H_path}

    def __len__(self):
        return len(self.paths_L)


#### Create Dataset and Dataloader

In [3]:
train_set = DatasetSR()
#train_sampler = DistributedSampler(train_set, shuffle=True, drop_last=True)
train_loader = DataLoader(train_set,
                          batch_size=8,
                          shuffle=True,
                          drop_last=True,
                          num_workers = 4,
                          pin_memory=True)

In [4]:
'''
from matplotlib import pyplot as plt
arr = x[1].permute(1,2,0)
#arr = torch.Tensor.numpy(x[1])
print(arr.shape)
plt.imshow(arr)

plt.imshow(x[1].permute(1, 2, 0),cmap='gray')
'''

"\nfrom matplotlib import pyplot as plt\narr = x[1].permute(1,2,0)\n#arr = torch.Tensor.numpy(x[1])\nprint(arr.shape)\nplt.imshow(arr)\n\nplt.imshow(x[1].permute(1, 2, 0),cmap='gray')\n"

### Wavelet Domain Deep CNN (W-SDNet)

In [5]:
import torch
import torch.nn as nn
#import tensorflow as tf
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
import torchvision


class Deep_Net(nn.Module):
    def __init__(self):
        super(Deep_Net, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

    def forward(self, x):
        out_1 = self.block(x)
        
        return out_1

class W_SDNet(nn.Module):
    def __init__(self):
        super(W_SDNet, self).__init__()
        self.head = nn.Sequential(
            nn.Conv2d(in_channels= 4, out_channels= 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        body = [Deep_Net() for _ in range(18)]       #Body has 18 layers
        self.body = nn.Sequential(*body)
        
        self.tail = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 4, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(4),
            nn.ReLU(),
        )
    def forward(self, x):
        #print(x.shape)
        copy = x.cpu()
        coeffs2 = pywt.dwt2(copy, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2
        LL = torch.from_numpy(LL)
        LH = torch.from_numpy(LH)
        HL = torch.from_numpy(HL)
        HH = torch.from_numpy(HH)
        concat_1 = torch.cat((LL,LH,HL,HH),1).to(device)
        output = self.head(concat_1)
        output = self.body(output)
        output = self.tail(output)
        output = concat_1 + output
        LL = output[:,0,:,:].cpu().detach().numpy()
        LH = output[:,1,:,:].cpu().detach().numpy()
        HL = output[:,2,:,:].cpu().detach().numpy()
        HH = output[:,3,:,:].cpu().detach().numpy()
        coeffs2 = LL, (LH, HL, HH)
        reconstructed = pywt.idwt2(coeffs2, 'bior1.3')
        reconstructed = reconstructed[:, None, :,:]

        return torch.from_numpy(reconstructed).to(device)

### Pixel Domain Deep CNN (P-SDNet) 

In [6]:
import torch
import torch.nn as nn
#import tensorflow as tf
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
import torchvision


class Deep_Net(nn.Module):
    def __init__(self):
        super(Deep_Net, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

    def forward(self, x):
        out_1 = self.block(x)
        
        return out_1

class P_SDNet(nn.Module):
    def __init__(self):
        super(P_SDNet, self).__init__()
        self.head = nn.Sequential(
            nn.Conv2d(in_channels= 4, out_channels= 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        body = [Deep_Net() for _ in range(18)]       #Body has 18 layers
        self.body = nn.Sequential(*body)
        
        self.tail = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 4, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(4),
            nn.ReLU(),
        )
    def forward(self, x):
        b,_,w,h = x.size()
        down = torch.reshape(x,(b,4, w//2, h//2))
        output = self.head(down)
        output = self.body(output)
        output = self.tail(output)
        output = down + output
        reconstructed = torch.reshape(output,(b,1, w, h))
        

        return reconstructed

### DPW_SD_Net Module

In [7]:
class DPW_SD_Net(nn.Module):
    def __init__(self):
        super(DPW_SD_Net, self).__init__()
        self.WSD = W_SDNet()
        self.PSD = P_SDNet()
        
        self.Patch_Reconstruction = nn.Sequential(nn.Conv2d(in_channels= 2, out_channels= 1, kernel_size=(1,1), stride=(1,1), padding=(0,0),bias = False)
                                      )      
        
    def forward(self, x):
        WSD = self.WSD(x)
        PSD = self.PSD(x)
        Concat = torch.cat((WSD,PSD),1)
        final = self.Patch_Reconstruction(Concat)
        
            
        
        
       
        return final


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DPW_SD_Net()
model.to(device)
co_ef = 1e-3
error = nn.MSELoss()

learning_rate = 0.1 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay= 0.0001) # Code to use the Adam Optimizer
scheduler = MultiStepLR(optimizer, milestones=[10 * 100 ,20 * 100 , 30 * 100, 40 * 100], gamma=0.5) #milestones=[Intended Epoch * Batch Size]
print(model)

DPW_SD_Net(
  (WSD): W_SDNet(
    (head): Sequential(
      (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (body): Sequential(
      (0): Deep_Net(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (1): Deep_Net(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (2): Deep_Net(
        (block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [9]:
#Template for loading presaved checkpoints for training
'''
#IAM_3 = IAM_Net()
saved_state_dict = torch.load('/notebooks/DPW_SD_Net_Files/models_JPEG_20/DPWS_JPEG_20_50000.pth')
model.load_state_dict(saved_state_dict)
#IAM_3 = IAM_3.to(device)
model.train()


#Load IAM Branch_1 Optimizer Checkpoint
saved_state_dict = torch.load('/notebooks/JPEG_Gray_IDPD_Files/models_1_channel/Optimizer300.pth')
optimizer.load_state_dict(saved_state_dict)
'''

"\n#Load IAM Branch_1 Optimizer Checkpoint\nsaved_state_dict = torch.load('/notebooks/JPEG_Gray_IDPD_Files/models_1_channel/Optimizer300.pth')\noptimizer.load_state_dict(saved_state_dict)\n"

In [10]:
import logging
logging.basicConfig(
        level=logging.DEBUG,
        format="%(asctime)s %(levelname)s %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        filename="/notebooks/DPW_SD_Net_Files/logs/train_DPWSD_JPEG_10.log",
    )

In [ ]:
from tqdm.auto import tqdm
num_epochs = 4000
count = 0 * 450
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.train()


for epoch in range(1,num_epochs): 
    for idx, train_data in tqdm(enumerate(train_loader)):
        labels = []
        L = train_data['L'].to(device)
        H = train_data['H'].to(device)
        
        
        
        
        
        
        output = model(L)
        
        

        loss = error(output, H)

        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        

        #Propagating the error backward
        loss.backward()

        # Optimizing the parameters
        optimizer.step()
        
        scheduler.step()
        
        count += 1

    if (count % 100 == 0):
        logging.info("Iteration: {}, Loss: {}%, epoch: {}, Learning Rate: {}".format(count, loss.data,epoch, scheduler.get_last_lr()))
        #scheduler.get_last_lr(), Learning Rate: {},
    if (count % 10000 == 0 and epoch != 0) or epoch ==3999 :
        torch.save(model.state_dict(), os.path.join("/notebooks/DPW_SD_Net_Files/models_JPEG_10", 'DPWS_JPEG_10_' + str(count) + '.pth'))
        torch.save(optimizer.state_dict(), os.path.join("/notebooks/DPW_SD_Net_Files/models_JPEG_10", 'Optimizer_' + str(count) + '.pth'))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]